# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import _data_utils
import _script_utils

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))

In [10]:

# df = utils.read_data_all()
proj = _data_utils.add_new_codes(proj)
    
    ## function that adds known agency name to df 
df = _script_utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [15]:
df.sample(3)

,unnamed:_0,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
4911,NaN,2022-11-21,ZS30,NaN,5017047,0119000106L,"35 INTERSECTIONS WITHIN THE CITY OF EUREKA PEDESTRIAN AND VEHICLE IMPROVEMENTS INCLUDING CROSSWALK MARKINGS, RAPID FLASHING BEACONS, CURB EXTENSIONS,",23,Cong Dist 2,Closed,"35 INTERSECTIONS WITHIN THE CITY OF EUREKA PEDESTRIAN AND VEHICLE IMPROVEMENTS INCLUDING CROSSWALK MARKINGS, RAPID FLASHING BEACONS, CURB EXTENSIONS, FLASHING BEACONS AT STOP SIGNS, SPEED FEEDBACK SIGNS, AND AUDIBLE PEDESTRIAN SIGNALS.",21,Safety,37976.2,37976.2,L5017NON-MPO,5017.0,Eureka,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO
2056,NaN,2022-06-02,M003,NaN,5059197,1012000274L,SB DALE RD. - SOUTH OF VENEMAN AVE. AND NORTH OF STANDIFORD AVE. INSTALL BUS TURNOUT (TC),99,Cong Dist 10,Closed,SB DALE RD. - SOUTH OF VENEMAN AVE. AND NORTH OF STANDIFORD AVE. INSTALL BUS TURNOUT (TC),15,Preliminary Engineering,6000.0,5312.0,L5059STANCOG,5059.0,Modesto,10.0,Stanislaus County,Stanislaus Council of Governments,Stanislaus Council Of Governments
5409,NaN,2022-12-22,Y001,National Highway Performance Program (NHPP),5919115,0313000226L,WATT AVE OVER DRY CREEK 0.4 MILES NORTH OF P.F.E. ROAD. BR.# 19C0084 BRIDGE REPLACEMENT,61,Cong Dist 4,Active,WATT AVE OVER DRY CREEK 0.4 MILES NORTH OF P.F.E. ROAD. BR.# 19C0084 BRIDGE REPLACEMENT,43,Utilities,1000000.0,522327.0,L5919SACOG,5919.0,Placer County,3.0,Placer County,Placer County Transportation Planning Agency,Sacramento Area Council Of Governments


## Test & Export

In [17]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

In [18]:
sorted(list(agg.project_title_new))

[' Added Roadway Capacity in Industry',
 ' Added Roadway Capacity in Kern County',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles',
 ' Added Roadway Capacity in Los Angeles County',
 ' Added Roadway Capacity in Los Angeles County & San Mateo County',
 ' Added Roadway Capacity in Madera County',
 ' Added Roadway Capacity in Marin County & San Francisco County',
 ' Added Roadway Capacity in Murrieta',
 ' Added Roadway Capacity in Needles',
 ' Added Roadway Capacity in Orange County',
 ' Added Roadway Capacity in Orange County',
 ' Added Roadway Capacity in Riverside County',
 ' Added Roadway Capacity in Riverside County',
 ' Added Roadway Capacity in San Bernardino County',
 ' Added Roadway Capacity in San Bernardino County',
 ' Added Roadway Capacity in San Bernardino County',
 ' Added Roadway Capacity in Santa Clara Valley Transportation Authority',
 ' Added Roadway Capacity in Santa Clara Valley Transportation Authority',
 ' Added Roadway Capacity 

In [19]:
len(agg)

3155

In [17]:
agg = utils.title_column_names(agg)

In [25]:
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_agg.csv")

In [19]:
###test full. PASS title, no concat and 1241 rows
full = utils.get_clean_data(df, full_or_agg = 'full')

In [20]:
full = utils.title_column_names(full)

In [27]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")